In [51]:
import zarr
import xarray as xr
from pyproj import CRS, Transformer


# group dir
tmp_zarr_group_dir  = "/playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210123_23z_anl/0_500m_above_ground/CANGLE"

# var dir
tmp_zarr_dir        = "/playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210123_23z_anl/0_500m_above_ground/CANGLE/0_500m_above_ground/CANGLE"

g = zarr.open(tmp_zarr_dir)

# proj y, proj x
g[:].shape

(1059, 1799)

In [ ]:
import s3fs
import xarray as xr


# for projecting zarr -> lat/lon
url = "s3://hrrrzarr/sfc/20210601/20210601_00z_anl.zarr"
fs = s3fs.S3FileSystem(anon=True)
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
chunk_index

<xarray.Dataset> Size: 46MB
Dimensions:     (x: 1799, y: 1059)
Coordinates:
  * x           (x) float64 14kB -2.698e+06 -2.695e+06 ... 2.693e+06 2.696e+06
  * y           (y) float64 8kB -1.587e+06 -1.584e+06 ... 1.584e+06 1.587e+06
Data variables:
    chunk_id    (x, y) object 15MB dask.array<chunksize=(1799, 1059), meta=np.ndarray>
    chunk_x     (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    chunk_y     (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    in_chunk_x  (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    in_chunk_y  (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    index_x     (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    index_y     (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    latitude    (y, x) float64 15MB dask.array<chunksize=(133, 450), meta=np.ndarray>
    longitude   (y, x) float64 15MB dask.array<chunksize=(133, 450), meta=np.ndarray>

In [61]:
chunk_index.longitude.shape

(1059, 1799)

In [30]:
import pandas as pd


df = pd.read_csv("/playpen-ssd/levi/ccrfcd-gauge-grids/data/2021-01-01_2025-07-25_gt_p1.csv")
df

,Unnamed: 0,gauge_idx,start_datetime_utc,end_datetime_utc,gauge_acc_in,mrms_q3evap_qpe,lat,lon
0,0,4564.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.00,0.000000,36.028250,-115.003639
1,1,4779.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.00,0.000000,36.021861,-114.958750
2,2,4709.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.00,0.003937,35.925000,-115.117000
3,3,4724.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.00,0.015748,35.939417,-115.077750
4,4,4564.0,2021-01-23 23:02:00+00:00,2021-01-24 00:02:00+00:00,0.00,0.000000,36.028250,-115.003639
...,...,...,...,...,...,...,...,...
778748,778748,4414.0,2023-08-20 21:52:00+00:00,2023-08-20 22:52:00+00:00,0.04,0.039370,36.031861,-115.316694
778749,778749,4419.0,2023-08-20 21:52:00+00:00,2023-08-20 22:52:00+00:00,0.04,0.051181,36.044722,-115.299722
778750,778750,4424.0,2023-08-20 21:52:00+00:00,2023-08-20 22:52:00+00:00,0.04,0.047244,36.087139,-115.305306
778751,778751,4429.0,2023-08-20 21:52:00+00:00,2023-08-20 22:52:00+00:00,0.04,0.039370,36.105639,-115.304028


In [ ]:
import zarr
from pathlib import Path
from datetime import datetime, timezone
from glob import glob


VARS_OF_INTEREST = [
    {"name": "DPT", "level": "2m_above_ground", "store": "sfc"},
    {"name": "PWAT", "level": "entire_atmosphere_single_layer", "store": "sfc"},
    {"name": "HGT", "level": "level_of_adiabatic_condensation_from_sfc", "store": "sfc"},
    {"name": "HGT", "level": "highest_tropospheric_freezing_level", "store": "sfc"},
    {"name": "APCP", "level": "surface", "store": "sfc"},
    {"name": "DPT", "level": "925mb", "store": "sfc"},
    {"name": "DPT", "level": "850mb", "store": "sfc"},
    {"name": "DPT", "level": "700mb", "store": "sfc"},
    {"name": "DPT", "level": "500mb", "store": "sfc"},
    {"name": "UGRD", "level": "850mb", "store": "sfc"},
    {"name": "VGRD", "level": "850mb", "store": "sfc"},
    {"name": "UGRD", "level": "700mb", "store": "sfc"},
    {"name": "VGRD", "level": "700mb", "store": "sfc"},
    {"name": "TMP", "level": "2m_above_ground", "store": "sfc"},
    {"name": "PRES", "level": "surface", "store": "sfc"},
    {"name": "HGT", "level": "700mb", "store": "sfc"},
    {"name": "HGT", "level": "850mb", "store": "sfc"},
    {"name": "TMP", "level": "700mb", "store": "sfc"},
    {"name": "TMP", "level": "850mb", "store": "sfc"},
    {"name": "TMP", "level": "500mb", "store": "sfc"},
    {"name": "SPFH", "level": "1000mb", "store": "prs"},
    {"name": "SPFH", "level": "925mb",  "store": "prs"},
    {"name": "SPFH", "level": "850mb",  "store": "prs"},
    {"name": "SPFH", "level": "700mb",  "store": "prs"},
    {"name": "RH",   "level": "925mb",  "store": "prs"},
    {"name": "RH",   "level": "850mb",  "store": "prs"},
    {"name": "RH",   "level": "700mb",  "store": "prs"},
]

HRRR_ENV_DIR = "/playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env"

# TZ assumed UTC
dt_fp_dict = {datetime.strptime(Path(fp).name, "%Y%m%d_%Hz_anl") : fp for fp in glob(f"{HRRR_ENV_DIR}/*")}


def get_hrrr_dir_path(dt: datetime) -> str | None:
    # truncate precision < hour
    y, m, d, h = dt.year, dt.month, dt.day, dt.hour
    truc_dt = datetime(year=y, month=m, day=d, hour=h)
    # return fp if in dict
    if truc_dt in dt_fp_dict: return dt_fp_dict[truc_dt]
    else: return None


def get_vertical_levels(hrrr_dir: str) -> list:
    return [Path(fp).name for fp in glob(f"{hrrr_dir}/*")]


def get_env_vars(hrrr_dir: str) -> dict:
    """
    Given a path to a HRRR dir (i.e., valid date/hr),

    Return
    ---
    ```
        { 
            {
                "var_name": ...,
                "level":, ...,
                "zgroup: ...,
                "zarr": ...,
            }
        }
    ```
    """

    evd = []
    env_var_zarrs = glob(f"{hrrr_dir}/*/*/*/*/.zarray")

    for fp in env_var_zarrs:

        p = Path(fp)
        # path to zarr dir for an atmospheric var
        env_var_zarr_fp  = p.parent
        env_var_group_fp = p.parent.parent.parent
        env_var_name     = p.parent.name
        level_name       = p.parent.parent.name

        for var_of_int in VARS_OF_INTEREST:
            lvl = var_of_int['level']
            nme = var_of_int['name']
            if lvl == level_name and nme == env_var_name:
                evd.append({
                    "level": level_name,
                    "var_name": env_var_name,
                    "var_group": zarr.open_group(env_var_group_fp),
                    "var_zarr": zarr.open_array(env_var_zarr_fp),
                })
                break

    return evd


dir_fp = get_hrrr_dir_path(datetime(year=2021, month=1, day=24, hour=0))
get_env_vars(dir_fp)

[{'level': 'surface',
  'var_name': 'PRES',
  'var_group': <Group file:///playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210124_00z_anl/surface/PRES>,
  'var_zarr': <Array file:///playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210124_00z_anl/surface/PRES/surface/PRES shape=(1059, 1799) dtype=float32>},
 {'level': 'level_of_adiabatic_condensation_from_sfc',
  'var_name': 'HGT',
  'var_group': <Group file:///playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210124_00z_anl/level_of_adiabatic_condensation_from_sfc/HGT>,
  'var_zarr': <Array file:///playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210124_00z_anl/level_of_adiabatic_condensation_from_sfc/HGT/level_of_adiabatic_condensation_from_sfc/HGT shape=(1059, 1799) dtype=float16>},
 {'level': '925mb',
  'var_name': 'DPT',
  'var_group': <Group file:///playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210124_00z_anl/925mb/DPT>,
  'var_zarr': <Array file:///playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210124_00z_an